# Visualize the manually constructed dataset of mining areas and ground truth masks

In [158]:
import geopandas as gpd
import leafmap
from shapely.ops import unary_union
from shapely.geometry import Point, mapping, box, shape
import shapely
import json

import sys
sys.path.append("..")

# Load the data
data = gpd.read_file("/workspaces/mine-segmentation/data/raw/mining_tiles_with_masks.gpkg")

data.head()

,tile_id,tile_bbox,sentinel_2_id,source_dataset,timestamp,geometry
0,621,"{""type"": ""Polygon"", ""coordinates"": [[[-63.3333...",S2B_MSIL2A_20190517T144739_R139_T20NMP_2020100...,tang,2024-06-19 09:07:50.341,"MULTIPOLYGON (((-63.41721 7.45592, -63.40836 7..."
1,742,"{""type"": ""Polygon"", ""coordinates"": [[[-58.1666...",S2A_MSIL2A_20190804T142801_R053_T21NUG_2020100...,maus,2024-06-19 09:10:00.813,"MULTIPOLYGON (((-58.3271 5.8973, -58.3273 5.89..."
2,1746,"{""type"": ""Polygon"", ""coordinates"": [[[87.0, 53...",S2A_MSIL2A_20191007T052711_R105_T45UVV_2020100...,maus,2024-06-19 09:11:02.652,"MULTIPOLYGON (((86.8675 53.567, 86.8708 53.566..."
3,2419,"{""type"": ""Polygon"", ""coordinates"": [[[118.0, -...",S2B_MSIL2A_20190112T021349_R060_T50KNV_2020100...,tang,2024-06-19 09:11:29.248,"MULTIPOLYGON (((117.83395 -23.29398, 117.83494..."
4,2205,"{""type"": ""Polygon"", ""coordinates"": [[[114.5, -...",S2A_MSIL2A_20190215T022801_R046_T50MKD_2020100...,tang,2024-06-19 09:11:58.095,"MULTIPOLYGON (((114.35284 -1.31591, 114.35241 ..."


In [141]:
# Create a Leaflet map
m = leafmap.Map()

# Add the GeoDataFrame to the map
m.add_gdf(data, layer_name="dataset")

# Display the map
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Visualize the global distribution of the tiles

In [143]:
# Using Markers on leafmap
def plot_tiles_on_basemap(gdf):
    import folium

    m = folium.Map()

    # Iterate over the tiles and add markers to the map
    for _, row in gdf.iterrows():
        lat = row.geometry.centroid.y
        lon = row.geometry.centroid.x
        folium.Marker([lat, lon]).add_to(m)

    # Display the map
    return m

plot_tiles_on_basemap(data)

# Test drawing bounding boxes around the polygons

In [145]:
def add_bounding_boxes(row):
    # apply buffer 
    row.geometry = row.geometry.buffer(0.0001)

    # create the unary union, i.e. merge touching polygons
    union = unary_union(row.geometry)

    if isinstance(union, shapely.geometry.multipolygon.MultiPolygon):
        # get list of bounding boxes
        bounding_boxes = [box(*geom.bounds) for geom in list(union.geoms)]
        
        # convert back to multipolygon
        bounding_boxes = shapely.geometry.MultiPolygon(bounding_boxes)
        
        # convert to GeoJSON string
        geojson_str = json.dumps(shapely.geometry.mapping(bounding_boxes))

        return geojson_str
    else:
        # Create a box from the bounding box
        bounding_box = box(*union.bounds)

        # Convert the bounding box to a GeoJSON string
        geojson_str = json.dumps(shapely.geometry.mapping(bounding_box))

        return geojson_str

# apply the function to the geodataframe
data['bounding_box'] = data.apply(add_bounding_boxes, axis=1)

data.head()

,tile_id,tile_bbox,sentinel_2_id,source_dataset,timestamp,geometry,bounding_box
0,621,"{""type"": ""Polygon"", ""coordinates"": [[[-63.3333...",S2B_MSIL2A_20190517T144739_R139_T20NMP_2020100...,tang,2024-06-19 09:07:50.341,"MULTIPOLYGON (((-63.41721 7.45592, -63.40836 7...","{""type"": ""Polygon"", ""coordinates"": [[[-63.3504..."
1,742,"{""type"": ""Polygon"", ""coordinates"": [[[-58.1666...",S2A_MSIL2A_20190804T142801_R053_T21NUG_2020100...,maus,2024-06-19 09:10:00.813,"MULTIPOLYGON (((-58.3271 5.8973, -58.3273 5.89...","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-5..."
2,1746,"{""type"": ""Polygon"", ""coordinates"": [[[87.0, 53...",S2A_MSIL2A_20191007T052711_R105_T45UVV_2020100...,maus,2024-06-19 09:11:02.652,"MULTIPOLYGON (((86.8675 53.567, 86.8708 53.566...","{""type"": ""MultiPolygon"", ""coordinates"": [[[[86..."
3,2419,"{""type"": ""Polygon"", ""coordinates"": [[[118.0, -...",S2B_MSIL2A_20190112T021349_R060_T50KNV_2020100...,tang,2024-06-19 09:11:29.248,"MULTIPOLYGON (((117.83395 -23.29398, 117.83494...","{""type"": ""MultiPolygon"", ""coordinates"": [[[[11..."
4,2205,"{""type"": ""Polygon"", ""coordinates"": [[[114.5, -...",S2A_MSIL2A_20190215T022801_R046_T50MKD_2020100...,tang,2024-06-19 09:11:58.095,"MULTIPOLYGON (((114.35284 -1.31591, 114.35241 ...","{""type"": ""Polygon"", ""coordinates"": [[[114.3529..."


# Visualize Bounding Boxes

In [146]:
ROW = 3

m = leafmap.Map(center = (data.iloc[ROW:ROW+1].geometry.centroid.y.values[0], data.iloc[ROW:ROW+1].geometry.centroid.x.values[0]), zoom = 12)
# add satellite
m.add_basemap("SATELLITE")
m.add_gdf(data.iloc[ROW:ROW+1], layer_name="dataset")

bbox = json.loads(data.iloc[ROW:ROW+1].bounding_box.values[0])

style = {
    "color": "green",
    "fillColor": "green",
    "fillOpacity": 0.0,
}
m.add_geojson(bbox, layer_name="tile", style = style)

# add tile
tile = json.loads(data.iloc[ROW:ROW+1].tile_bbox.values[0])

style_tile = {
    "color": "orange",
    "fillColor": "orange",
    "fillOpacity": 0.0,
}

m.add_geojson(tile, layer_name="tile", style = style_tile)

m

Map(center=[-23.310089923561, 117.85170907428787], controls=(ZoomControl(options=['position', 'zoom_in_text', …

KeyError: 'properties'

KeyError: 'properties'

KeyError: 'properties'

KeyError: 'properties'

KeyError: 'properties'

KeyError: 'properties'

KeyError: 'properties'

# Load processed dataset

In [159]:
# list layernames 
gpd.list_layers("/workspaces/mine-segmentation/data/processed/mining_tiles_with_masks_and_bounding_boxes.gpkg")

,name,geometry_type
0,tiles,Polygon
1,masks,MultiPolygon
2,bounding_boxes,MultiPolygon


In [160]:
# load the processed dataset
tiles = gpd.read_file("/workspaces/mine-segmentation/data/processed/mining_tiles_with_masks_and_bounding_boxes.gpkg", layer="tiles")
masks = gpd.read_file("/workspaces/mine-segmentation/data/processed/mining_tiles_with_masks_and_bounding_boxes.gpkg", layer="masks")
bounding_boxes = gpd.read_file("/workspaces/mine-segmentation/data/processed/mining_tiles_with_masks_and_bounding_boxes.gpkg", layer="bounding_boxes")
tiles.head()

,tile_id,sentinel_2_id,source_dataset,timestamp,geometry
0,621,S2B_MSIL2A_20190517T144739_R139_T20NMP_2020100...,tang,2024-06-19 09:07:50.341,"POLYGON ((-63.33333 7.33333, -63.33333 7.5, -6..."
1,742,S2A_MSIL2A_20190804T142801_R053_T21NUG_2020100...,maus,2024-06-19 09:10:00.813,"POLYGON ((-58.16667 5.83333, -58.16667 6, -58...."
2,1746,S2A_MSIL2A_20191007T052711_R105_T45UVV_2020100...,maus,2024-06-19 09:11:02.652,"POLYGON ((87 53.5, 87 53.66667, 86.83333 53.66..."
3,2419,S2B_MSIL2A_20190112T021349_R060_T50KNV_2020100...,tang,2024-06-19 09:11:29.248,"POLYGON ((118 -23.33333, 118 -23.16667, 117.83..."
4,2205,S2A_MSIL2A_20190215T022801_R046_T50MKD_2020100...,tang,2024-06-19 09:11:58.095,"POLYGON ((114.5 -1.33333, 114.5 -1.16667, 114...."


## Visualize the processed dataset

In [162]:
ROW = 4

m = leafmap.Map(center = (tiles.iloc[ROW:ROW+1].geometry.centroid.y.values[0], tiles.iloc[ROW:ROW+1].geometry.centroid.x.values[0]), zoom = 12)
# add satellite
m.add_basemap("SATELLITE")

style_tile = {
    "color": "orange",
    "fillColor": "orange",
    "fillOpacity": 0.0,
}

style_bbox = {
    "color": "green",
    "fillColor": "green",
    "fillOpacity": 0.0,
}

m.add_gdf(tiles.iloc[ROW:ROW+1], layer_name="tiles", style=style_tile)
m.add_gdf(masks.iloc[ROW:ROW+1], layer_name="masks")
m.add_gdf(bounding_boxes.iloc[ROW:ROW+1], layer_name="bboxes", style=style_bbox)
m

Map(center=[-1.249999999999993, 114.41666666666664], controls=(ZoomControl(options=['position', 'zoom_in_text'…